In [64]:
import requests, json
import pandas as pd
BASE_URL = "https://api.weather.gov/gridpoints/GSP/40,68/"
USER_AGENT = "weather-learner/1.0"
CACHED_DATA_FILE = "forecast.json" 

def fetch_forecast() -> dict[str, dict]:
    return requests.get(
        BASE_URL + "forecast",
        headers={"User-Agent": USER_AGENT}
    ).json()

def load_cached_data() -> dict[str,dict]:
    try:
        with open(CACHED_DATA_FILE, "r") as f:
            import json
            return json.load(f)
    except FileNotFoundError:
        return {}

def cache_forecast(forecast_data) -> None:
    with open(CACHED_DATA_FILE, "w") as f:
        json.dump(forecast_data, f, indent=4)
    print("Forecast data saved to forecast.json")


In [65]:
forecast_data_raw: dict[str, dict] = load_cached_data()
forecast_data_df = pd.DataFrame(forecast_data_raw['properties']['periods']) # type: ignore
forecast_data_df = forecast_data_df.set_index("name")

In [66]:
forecast_data_df_mod = forecast_data_df.drop(columns=['number', 'startTime', 'endTime', 'isDaytime', 'temperatureUnit', 'icon', 'temperatureTrend'])
forecast_data_df_mod["probabilityOfPrecipitation"] = forecast_data_df["probabilityOfPrecipitation"].map(lambda x: x.get("value") if isinstance(x, dict) else None)

In [67]:
forecast_data_df_mod

,temperature,probabilityOfPrecipitation,windSpeed,windDirection,shortForecast,detailedForecast
name,,,,,,
Juneteenth,79,60,9 mph,WSW,Showers And Thunderstorms Likely,Showers and thunderstorms likely. Mostly cloud...
Tonight,60,44,2 to 9 mph,WNW,Chance Showers And Thunderstorms then Patchy Fog,A chance of showers and thunderstorms before m...
Friday,82,3,5 mph,N,Patchy Fog then Sunny,"Patchy fog before 10am. Sunny, with a high nea..."
Friday Night,60,3,1 to 5 mph,SE,Mostly Clear,"Mostly clear, with a low around 60. Southeast ..."
Saturday,86,6,3 mph,S,Sunny,"Sunny, with a high near 86. South wind around ..."
Saturday Night,62,5,3 mph,SSW,Mostly Clear,"Mostly clear, with a low around 62."
Sunday,88,12,3 mph,NNW,Sunny,"Sunny, with a high near 88."
Sunday Night,65,10,3 mph,NW,Mostly Clear,"Mostly clear, with a low around 65."
Monday,89,20,1 to 5 mph,N,Sunny then Slight Chance Showers And Thunderst...,A slight chance of showers and thunderstorms a...
